# Generative nettverk

Rekurrente nevrale nettverk (RNNs) og deres gatede cellevarianter som Long Short Term Memory Cells (LSTMs) og Gated Recurrent Units (GRUs) har gitt en mekanisme for språkmodellering, dvs. de kan lære ordrekkefølge og gi prediksjoner for neste ord i en sekvens. Dette gjør det mulig å bruke RNNs til **generative oppgaver**, som vanlig tekstgenerering, maskinoversettelse og til og med bildetekstgenerering.

I RNN-arkitekturen vi diskuterte i forrige enhet, produserte hver RNN-enhet neste skjulte tilstand som et output. Imidlertid kan vi også legge til et annet output til hver rekurrente enhet, som gjør det mulig for oss å generere en **sekvens** (som er like lang som den opprinnelige sekvensen). Videre kan vi bruke RNN-enheter som ikke tar imot en input på hvert steg, men kun tar en initial tilstandsvektor og deretter produserer en sekvens av outputs.

I denne notatboken vil vi fokusere på enkle generative modeller som hjelper oss med å generere tekst. For enkelhets skyld skal vi bygge et **tegn-nivå nettverk**, som genererer tekst bokstav for bokstav. Under trening må vi ta et tekstkorpus og dele det opp i tegnsekvenser.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

## Bygge opp tegnordforråd

For å bygge et tegnbasert generativt nettverk, må vi dele opp tekst i individuelle tegn i stedet for ord. `TextVectorization`-laget vi har brukt tidligere kan ikke gjøre dette, så vi har to alternativer:

* Laste inn tekst manuelt og gjøre tokenisering 'for hånd', som vist i [dette offisielle Keras-eksempelet](https://keras.io/examples/generative/lstm_character_level_text_generation/)
* Bruke `Tokenizer`-klassen for tegnbasert tokenisering.

Vi velger det andre alternativet. `Tokenizer` kan også brukes til å tokenisere til ord, så det skal være ganske enkelt å bytte fra tegnbasert til ordnivå-tokenisering.

For å gjøre tegnbasert tokenisering, må vi sende inn parameteren `char_level=True`:


In [2]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts([x['title'].numpy().decode('utf-8') for x in ds_train])

Vi ønsker også å bruke en spesiell token for å markere **slutt på sekvens**, som vi vil kalle `<eos>`. La oss legge den til manuelt i vokabularet:


In [3]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1

In [4]:
tokenizer.texts_to_sequences(['Hello, world!'])

[[48, 2, 10, 10, 5, 44, 1, 25, 5, 8, 10, 13, 78]]

## Trene en generativ RNN til å lage titler

Måten vi skal trene RNN til å generere nyhetstitler på er som følger. For hvert steg tar vi én tittel, som mates inn i en RNN, og for hvert inndata-tegn ber vi nettverket om å generere neste utdata-tegn:

![Bilde som viser et eksempel på RNN-generering av ordet 'HELLO'.](../../../../../translated_images/no/rnn-generate.56c54afb52f9781d.webp)

For det siste tegnet i sekvensen vår ber vi nettverket om å generere `<eos>`-token.

Den største forskjellen med den generative RNN-en vi bruker her er at vi tar utdata fra hvert steg i RNN-en, og ikke bare fra den siste cellen. Dette kan oppnås ved å spesifisere `return_sequences`-parameteren til RNN-cellen.

Dermed, under treningen, vil inndata til nettverket være en sekvens av kodede tegn av en viss lengde, og utdata vil være en sekvens av samme lengde, men forskjøvet med ett element og avsluttet med `<eos>`. Minibatch vil bestå av flere slike sekvenser, og vi må bruke **padding** for å justere alle sekvensene.

La oss lage funksjoner som vil transformere datasettet for oss. Fordi vi ønsker å padde sekvenser på minibatch-nivå, vil vi først batch'e datasettet ved å kalle `.batch()`, og deretter `map` det for å utføre transformasjonen. Så transformasjonsfunksjonen vil ta en hel minibatch som parameter:


In [5]:
def title_batch(x):
    x = [t.numpy().decode('utf-8') for t in x]
    z = tokenizer.texts_to_sequences(x)
    z = tf.keras.preprocessing.sequence.pad_sequences(z)
    return tf.one_hot(z,vocab_size), tf.one_hot(tf.concat([z[:,1:],tf.constant(eos_token,shape=(len(z),1))],axis=1),vocab_size)

Noen viktige ting vi gjør her:
* Vi starter med å hente ut selve teksten fra streng-tensoren
* `text_to_sequences` konverterer listen av strenger til en liste av heltallstensorer
* `pad_sequences` fyller ut disse tensorene til deres maksimale lengde
* Til slutt one-hot-koder vi alle tegnene, og utfører også forskyvning og legger til `<eos>`. Vi skal snart se hvorfor vi trenger one-hot-kodede tegn

Denne funksjonen er imidlertid **Pythonisk**, dvs. den kan ikke automatisk oversettes til Tensorflow sin beregningsgraf. Vi vil få feil hvis vi prøver å bruke denne funksjonen direkte i `Dataset.map`-funksjonen. Vi må omslutte dette Pythoniske kallet ved å bruke `py_function`-innpakningen:


In [6]:
def title_batch_fn(x):
    x = x['title']
    a,b = tf.py_function(title_batch,inp=[x],Tout=(tf.float32,tf.float32))
    return a,b

> **Merk**: Å skille mellom Pythonic- og Tensorflow-transformasjonsfunksjoner kan virke litt for komplisert, og du lurer kanskje på hvorfor vi ikke transformerer datasettet ved hjelp av standard Python-funksjoner før vi sender det til `fit`. Selv om dette absolutt kan gjøres, har det en stor fordel å bruke `Dataset.map`, fordi datatransformasjonsrørledningen utføres ved hjelp av Tensorflows beregningsgraf, som drar nytte av GPU-beregninger og minimerer behovet for å overføre data mellom CPU/GPU.

Nå kan vi bygge vårt generatornettverk og starte treningen. Det kan baseres på hvilken som helst rekurrent celle som vi diskuterte i forrige enhet (simple, LSTM eller GRU). I vårt eksempel vil vi bruke LSTM.

Siden nettverket tar tegn som input, og vokabularstørrelsen er ganske liten, trenger vi ikke en embedding-lag; én-hot-kodet input kan sendes direkte inn i LSTM-cellen. Utgangslaget vil være en `Dense`-klassifiserer som konverterer LSTM-utgangen til én-hot-kodede token-numre.

I tillegg, siden vi jobber med sekvenser av variabel lengde, kan vi bruke `Masking`-laget for å lage en maske som ignorerer den utfylte delen av strengen. Dette er ikke strengt nødvendig, fordi vi ikke er veldig interessert i alt som går utover `<eos>`-tokenet, men vi vil bruke det for å få litt erfaring med denne lagtypen. `input_shape` vil være `(None, vocab_size)`, hvor `None` indikerer sekvensen med variabel lengde, og utgangsformen er også `(None, vocab_size)`, som du kan se fra `summary`:


In [7]:
model = keras.models.Sequential([
    keras.layers.Masking(input_shape=(None,vocab_size)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy')

model.fit(ds_train.batch(8).map(title_batch_fn))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 84)          0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         109056    
_________________________________________________________________
dense (Dense)                (None, None, 84)          10836     
Total params: 119,892
Trainable params: 119,892
Non-trainable params: 0
_________________________________________________________________
15000/15000 [==============================] - 229s 15ms/step - loss: 1.5385


## Generere output

Nå som vi har trent modellen, ønsker vi å bruke den til å generere noe output. Først og fremst trenger vi en måte å dekode tekst representert av en sekvens av token-numre. For å gjøre dette, kunne vi bruke funksjonen `tokenizer.sequences_to_texts`; men den fungerer ikke godt med tokenisering på tegnnivå. Derfor vil vi ta en ordbok med tokens fra tokenizer (kalt `word_index`), bygge et reversert kart, og skrive vår egen dekodingsfunksjon:


In [10]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map[t] for t in x])

Nå skal vi generere. Vi starter med en streng `start`, koder den til en sekvens `inp`, og deretter kaller vi nettverket vårt på hvert steg for å finne neste tegn.

Utdataene fra nettverket `out` er en vektor med `vocab_size` elementer som representerer sannsynlighetene for hver token, og vi kan finne nummeret på den mest sannsynlige tokenen ved å bruke `argmax`. Deretter legger vi til dette tegnet i den genererte listen av tokens og fortsetter genereringen. Denne prosessen med å generere ett tegn gjentas `size` ganger for å generere ønsket antall tegn, og vi avslutter tidligere hvis `eos_token` oppdages.


In [12]:
def generate(model,size=100,start='Today '):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

'Today #39;s lead to strike for the strike for the strike for the strike (AFP)'

## Eksempel på output under trening

Siden vi ikke har noen nyttige målinger som *nøyaktighet*, er den eneste måten vi kan se at modellen vår blir bedre på, ved å **eksemplifisere** genererte strenger under trening. For å gjøre dette, vil vi bruke **callbacks**, altså funksjoner som vi kan sende til `fit`-funksjonen, og som vil bli kalt med jevne mellomrom under trening.


In [13]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model))
)

model.fit(ds_train.batch(8).map(title_batch_fn),callbacks=[sampling_callback],epochs=3)

Epoch 1/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.2703
Today #39;s a lead in the company for the strike
Epoch 2/3
15000/15000 [==============================] - 227s 15ms/step - loss: 1.2057
Today #39;s the Market Service on Security Start (AP)
Epoch 3/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.1752
Today #39;s a line on the strike to start for the start


Dette eksempelet genererer allerede ganske god tekst, men det kan forbedres på flere måter:

* **Mer tekst**. Vi har kun brukt titler for oppgaven vår, men du kan eksperimentere med fullstendig tekst. Husk at RNN-er ikke er så gode til å håndtere lange sekvenser, så det gir mening enten å dele dem opp i kortere setninger, eller alltid trene på en fast sekvenslengde med en forhåndsdefinert verdi `num_chars` (for eksempel 256). Du kan prøve å endre eksempelet ovenfor til en slik arkitektur, ved å bruke [offisiell Keras-veiledning](https://keras.io/examples/generative/lstm_character_level_text_generation/) som inspirasjon.

* **Flerlags LSTM**. Det kan være fornuftig å prøve 2 eller 3 lag med LSTM-celler. Som nevnt i forrige enhet, trekker hvert lag av LSTM ut visse mønstre fra tekst, og i tilfelle av en generator på tegnnivå kan vi forvente at det laveste LSTM-laget er ansvarlig for å trekke ut stavelser, og de høyere lagene - for ord og ordkombinasjoner. Dette kan enkelt implementeres ved å sende parameteren for antall lag til LSTM-konstruktøren.

* Du kan også eksperimentere med **GRU-enheter** og se hvilke som gir bedre resultater, samt med **forskjellige størrelser på skjulte lag**. For store skjulte lag kan føre til overtilpasning (f.eks. at nettverket lærer nøyaktig tekst), mens mindre størrelser kanskje ikke gir gode resultater.


## Myk tekstgenerering og temperatur

I den tidligere definisjonen av `generate` valgte vi alltid tegnet med høyest sannsynlighet som neste tegn i den genererte teksten. Dette førte ofte til at teksten "gikk i sirkel" og gjentok de samme tegnsekvensene igjen og igjen, som i dette eksempelet:
```
today of the second the company and a second the company ...
```

Men hvis vi ser på sannsynlighetsfordelingen for neste tegn, kan det hende at forskjellen mellom de høyeste sannsynlighetene ikke er så stor, for eksempel kan ett tegn ha sannsynlighet 0,2, mens et annet har 0,19, osv. For eksempel, når vi ser etter neste tegn i sekvensen '*play*', kan neste tegn like gjerne være et mellomrom eller **e** (som i ordet *player*).

Dette fører oss til konklusjonen at det ikke alltid er "rettferdig" å velge tegnet med høyest sannsynlighet, fordi det å velge det nest høyeste fortsatt kan føre til meningsfull tekst. Det er klokere å **samle** tegn fra sannsynlighetsfordelingen som er gitt av nettverksutgangen.

Denne sampling-prosessen kan gjøres ved hjelp av funksjonen `np.multinomial`, som implementerer den såkalte **multinomiale fordelingen**. En funksjon som implementerer denne **myke** tekstgenereringen er definert nedenfor:


In [33]:
def generate_soft(model,size=100,start='Today ',temperature=1.0):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            if nc==eos_token:
                break
            chars.append(nc)
            inp = inp+[nc]
        return decode(chars)

words = ['Today ','On Sunday ','Moscow, ','President ','Little red riding hood ']
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"\n--- Temperature = {i}")
    for j in range(5):
        print(generate_soft(model,size=300,start=words[j],temperature=i))


--- Temperature = 0.3
Today #39;s strike #39; to start at the store return
On Sunday PO to Be Data Profit Up (Reuters)
Moscow, SP wins straight to the Microsoft #39;s control of the space start
President olding of the blast start for the strike to pay &lt;b&gt;...&lt;/b&gt;
Little red riding hood ficed to the spam countered in European &lt;b&gt;...&lt;/b&gt;

--- Temperature = 0.8
Today countie strikes ryder missile faces food market blut
On Sunday collores lose-toppy of sale of Bullment in &lt;b&gt;...&lt;/b&gt;
Moscow, IBM Diffeiting in Afghan Software Hotels (Reuters)
President Ol Luster for Profit Peaced Raised (AP)
Little red riding hood dace on depart talks #39; bank up

--- Temperature = 1.0
Today wits House buiting debate fixes #39; supervice stake again
On Sunday arling digital poaching In for level
Moscow, DS Up 7, Top Proble Protest Caprey Mamarian Strike
President teps help of roubler stepted lessabul-Dhalitics (AFP)
Little red riding hood signs on cash in Carter-youb

---

KeyError: 0

Vi har introdusert en ekstra parameter kalt **temperatur**, som brukes til å indikere hvor strengt vi skal holde oss til den høyeste sannsynligheten. Hvis temperaturen er 1.0, gjør vi rettferdig multinomial sampling, og når temperaturen går mot uendelig - blir alle sannsynligheter like, og vi velger neste tegn tilfeldig. I eksempelet nedenfor kan vi observere at teksten blir meningsløs når vi øker temperaturen for mye, og den ligner "syklisk" hard-generert tekst når den nærmer seg 0.



---

**Ansvarsfraskrivelse**:  
Dette dokumentet er oversatt ved hjelp av AI-oversettelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selv om vi streber etter nøyaktighet, vær oppmerksom på at automatiske oversettelser kan inneholde feil eller unøyaktigheter. Det originale dokumentet på sitt opprinnelige språk bør anses som den autoritative kilden. For kritisk informasjon anbefales profesjonell menneskelig oversettelse. Vi er ikke ansvarlige for misforståelser eller feiltolkninger som oppstår ved bruk av denne oversettelsen.
